# G) Gauging in 3D magnetostatics

- **<font color='green'>[RUN & OBSERVE]</font>** $\rightarrow$ the cell should be run directly without modification
- **<font color='orange'>[RUN & PLAY]</font>** $\rightarrow$ the cell can be run directly, but some parameters should be changed interactively
- **<font color='red'>[FILL & RUN]</font>**    $\rightarrow$ the cell should be filled before being run
- **<font color='magenta'>[FILL & PLAY]</font>** $\rightarrow$ the cell should be filled, and then some parameters should be changed interactively.

_______
Solving the magnetostatics problem consists in finding $a\in H^0(\text{curl},\Omega)$, such that
$$ \forall v\in H^0(\text{curl},\Omega), \quad \int_\Omega \text{curl}~v \cdot (\nu \; \text{curl}~a) = \int_\Omega v \cdot j $$

Also, $\text{curl}(\text{grad}(\cdot))$ is always $0$. So for a solution $a$, we can find another solution $\tilde{a} = a + \text{grad}~u$ for any differentiable scalar field $u$.

We can see that setting proper boundary condition won't fix the issue this time, since $\text{grad}~u$ can also be defined inside $\Omega$.

## 1) Geometry and properties
We start by defining the 3D geometry and mesh.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from ngsolve.webgui import Draw
from utils.myGeometries import geometryCoil
from ngsolve import Mesh

Omega = Mesh(geometryCoil(splitting = 1).GenerateMesh(maxh=0.02))
print(f"Mesh statistics : {Omega.nv} vertices, {Omega.nedge} edges, {Omega.ne} elements")
print(f"Boundaries names : {Omega.GetBoundaries()[-6:]}")
print(f"Region names : {Omega.GetMaterials()}")
Draw(Omega, clipping = {"x" : 0, "y":1, "z" : -1, "dist":0.035})

### b) Material properties

Then, we define reluctivity and current density fields. We assume iron is linear with $\mu_r = 1000$ to avoid long computation time, and $J = 1A/mm^2$.

|**<font color='red'>[FILL & RUN]</font>** |
|---|

In [ ]:
from ngsolve import pi, Normalize, CoefficientFunction, x, y
J = 1e6
mu0 = 4e-7 * pi
nu = ............................
j =  ............................

#-----------------------------------------------------------------------------------------------
# Show the current
from ngsolve import L2, GridFunction
jL2 = GridFunction(L2(Omega, dim = 3))
jL2.Set(j)
Draw(jL2, Omega, vectors={"grid_size" : 50} , clipping = { "pnt" : (0,0,0), "vec" : (0,0,-1) })

## 2) Solver

### a) Wrapper
We then define a shorthand function for the solver:
- "direct" : use sparse cholesky decomposition (only for symmetric positive definite matrix)
- "cg" : use conjugate gradient solver (only for symmetric positive definite matrix)
- "gmres" : use gmres solver (for any matrix)

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from utils.mySolvers import mySolver
help(mySolver)

### b) Naive resolution

If we don't know that the 3D magnetostatics problem is singular, one might try to solve it anyway. Let's see what happens.

|**<font color='magenta'>[FILL & PLAY]</font>** |
|---|

In [ ]:
from ngsolve import HCurl

dirichletLabel = "dOmega_back|dOmega_forward|dOmega_left|dOmega_right|dOmega_up|dOmega_bottom"

fes = ............................
a, v = fes.TnT()


from ngsolve import BilinearForm, LinearForm, dx, curl
A = BilinearForm(fes)
A += ............................

l = LinearForm(fes)
l += ............................

#---------------------------------------------------------------------------------------
# Solve & draw
sol =  mySolver(A, l, type = ............................, preconditioner = ............................)
clipinfo= { "pnt" : (0,0,0), "vec" : (0,1,0) }
Draw(curl(sol), Omega, vectors={"grid_size" : 50}, clipping = clipinfo)

In general it doesn't work, or is very slow. The matrix is singular, we need a special procedure.

__________
## 3) Gauging

### a) Add a small mass term

We simply add a mass term $\epsilon \nu a \cdot v$ to the integral, with $\epsilon <<1$. It is called a mass term with analogy to mechanics where a similar term is associated to the mass.

|**<font color='magenta'>[FILL & PLAY]</font>** |
|---|

In [ ]:
fes = ............................,
a, v = fes.TnT()

eps = ............................,
Amass = BilinearForm(fes)
Amass += ............................,

l = LinearForm(fes)
l += ............................,


#---------------------------------------------------------------------------------------
# Draw
clipinfo= { "pnt" : (0,0,0), "vec" : (0,1,0) }
sol =  mySolver(............................,)
Draw(curl(sol), Omega, vectors={"grid_size" : 50}, clipping = clipinfo)

It works perfectly fine with the direct solver : the matrix is now invertible. 
However, it works not so well with iterative solvers that are very sensitive to the condition number (very bad if the terms in the matrix are very different... which is the essence of this method). 

You can try to use a preconditioner (`local` for instance) or increase the number of iterations to see what happens.

_______
### b) Coulomb gauge

Another idea is to add another constraint equation to impose uniqueness of $a$ through imposing its divergence to 0 (or a value depending on a Lagrange multiplier, in red).
We can show that the weak form can be written as a *mixed* formulation :

Find $a, \lambda \in H_0(\text{curl},\Omega) \times H^1_0(\Omega)$

$$ \left \{
\begin{array}{llll}
\forall a \in H_0(\text{curl},\Omega), & \displaystyle \int_\Omega \text{curl}\;v \cdot (\nu \text{curl}~a) &+ \displaystyle  \int_\Omega \text{grad}\;\lambda \cdot v &= \displaystyle  \int_\Omega v\cdot j \\
\forall \mu \in H^1_0(\Omega), & \displaystyle  \int_\Omega \text{grad}\;\mu \cdot a & + \textcolor{red}{\displaystyle \int_\Omega \mu \lambda }&= 0\\
\end{array} \right.$$

We can define coumpound spaces in NGSolve simply by
- `compoundSpace = fes1 * fes2`
- `a, lam = compoundSpace.TrialFunction()`
- `v, mu = compoundSpace.TestFunction()`

|**<font color='magenta'>[FILL & PLAY]</font>** |
|---|

In [ ]:
from ngsolve import H1, grad

fes = ............................,
a, lam = fes.TrialFunction()
v, mu = fes.TestFunction()

ACoulomb = BilinearForm(fes)
ACoulomb += ............................,

l = LinearForm(fes)
l += ............................,

#---------------------------------------------------------------------------------------
# Solve & draw
sol =  mySolver(............................,)
clipinfo= { "pnt" : (0,0,0), "vec" : (0,1,0) }
Draw(curl(sol.components[0]), Omega, vectors={"grid_size" : 50}, clipping = clipinfo)

The matrix is not positive definite anymore! It has a saddle point structure.
see [https://jschoeberl.github.io/iFEM/saddlepoint/structure.html](https://jschoeberl.github.io/iFEM/saddlepoint/structure.html)

So neither Cholesky decomposition neither conjugate gradient can work anymore (we can use other direct solvers as Pardiso, unfortunately not installed in the online version); also, "local" preconditioner will lead to problems. There are some dedicated preconditioners but their study is out of the scope of this tutorial.

$\rightarrow $ Adding the mass term $\int_\Omega \lambda \mu$ to the 2nd equation helps (sets the divergence to a different value, but the method is still exact and is not an approximation). 

To sum up, Coulomb gauge is not so simple to solve, and add more unknowns to the problem. But there are other ways!

_____
### c) Conforming right-hand side.

Actually, the current should be divergence free. It is not the case at the discrete level; but we can enforce this easily by expressing the current density as $j = \text{curl}\;t$. This can be done by solving first an auxiliary problem (that naturally have a compatible RHS):

$$ \text{Find}~ t \in H_0(\text{curl},\Omega), ~\text{such that}~ \forall v \in H_0(\text{curl},\Omega), \quad \int_\Omega \text{curl}~ v \cdot \text{curl}~ t = \int_\Omega \text{curl}~v \cdot j $$

|**<font color='red'>[FILL & RUN]</font>** |
|---|


In [ ]:
fesJ = ............................,
t, v = fesJ.TnT()
Aj = ............................,
fj = ............................,
#---------------------------------------------------------------------------------------
# Solve & draw
sol_t =  mySolver(............................,)
clipinfo= { "pnt" : (0,0,0), "vec" : (0,1,0) }
Draw(curl(sol_t), Omega, vectors={"grid_size" : 50} , clipping = { "pnt" : (0,0,0), "vec" : (0,0,-1) })

Then, we solve the 3D magnetostatics : 
$$ \forall v\in H^0(\text{curl},\Omega), \quad \int_\Omega \text{curl}~v \cdot (\nu \; \text{curl}~a) = \int_\Omega v \cdot \text{curl}\; t $$
|**<font color='magenta'>[FILL & PLAY]</font>** |
|---|

In [ ]:
fesA = ............................,
a, v = fesA.TnT()
Aa = ............................,
fa = ............................,
#---------------------------------------------------------------------------------------
# Solve & draw
sol_a =  mySolver(............................,)
clipinfo= { "pnt" : (0,0,0), "vec" : (0,1,0) }
Draw(curl(sol_a),Omega, vectors={"grid_size" : 50}, clipping = clipinfo)

Both problem are super fast to solve with conjugate gradient (maybe with a simple Jacobi preconditioner `local`).

_____
### d) Tree-cotree
Another possibility is to remove the redundant DoFs (edges) that are on a spanning tree, and solve the problem on the remaining ones (co-tree). It is therefore possible to form include any faces of the mesh (carrying an elementary flux) in a loop with a single DoF, making the problem invertible.

We can use `CoTreeBitArray` from `ngcotree` that does exactly that. 

Usage :
`fes.FreeDofs()[:] = CoTreeBitArray(Omega, fes)` with `fes` a `HCurl` finite element space.

|**<font color='magenta'>[FILL & PLAY]</font>** |
|---|

In [ ]:
from utils.ngcotree import CoTreeBitArray

fes = ............................,
fes.FreeDofs()[:] = CoTreeBitArray(Omega, fes)

a, v = fes.TnT()

Acotree = BilinearForm(fes)
Acotree += ............................,

l = LinearForm(fes)
l += ............................,

#---------------------------------------------------------------------------------------
# Solve & draw
sol =  mySolver(............................,)
clipinfo= { "pnt" : (0,0,0), "vec" : (0,1,0) }
Draw(curl(sol), Omega, vectors={"grid_size" : 50}, clipping = clipinfo)

Now, even direct solvers like `sparsecholesky` converges! However we notice that the iterative ones need a lot of iterations.